# Topic 3: Conditional Edges

Learn how to add decision-making capabilities to your graphs using conditional edges. This allows your workflows to adapt dynamically based on state.

## Learning Objectives

- Implement conditional routing
- Create decision nodes
- Build branching workflows
- Use state for routing decisions

In [1]:
# Setup
import os
import getpass
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(model="claude-3-5-sonnet-20241022")
print("✓ Setup complete!")

✓ Setup complete!


## Example: Customer Support Router

Let's build a customer support system that routes queries to different handlers based on their type.

In [2]:
# Define our state
class SupportState(TypedDict):
    query: str
    category: str
    response: str
    urgency: str

print("✓ SupportState defined")

✓ SupportState defined


## Step 1: Classify the Query

First, we need a node that determines the type of customer query:

In [ ]:
def classify_query(state: SupportState) -> SupportState:
    """Classify the customer query into a category."""
    print("🔍 Classifying query...")
    
    prompt = f"""Classify this customer support query into ONE of these categories:
    - technical (technical issues, bugs, errors)
    - billing (payments, invoices, pricing)
    - general (general questions, information requests)
    
    Also rate urgency as: low, medium, or high
    
    Query: {state['query']}
    
    Respond in this format:
    Category: [category]
    Urgency: [urgency]"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    content = response.content.lower()
    
    # Parse the response
    category = "general"
    urgency = "medium"
    
    if "category: technical" in content:
        category = "technical"
    elif "category: billing" in content:
        category = "billing"
    elif "category: general" in content:
        category = "general"
    
    if "urgency: high" in content:
        urgency = "high"
    elif "urgency: low" in content:
        urgency = "low"
    
    print(f"   Category: {category}")
    print(f"   Urgency: {urgency}")
    
    return {
        "category": category,
        "urgency": urgency
    }

print("✓ Classifier node created")

## Step 2: Create Specialized Handlers

Now we'll create different nodes for handling each category:

In [ ]:
def handle_technical(state: SupportState) -> SupportState:
    """Handle technical support queries."""
    print("🔧 Handling technical issue...")
    
    prompt = f"""You are a technical support specialist. Help with this issue:
    
    {state['query']}
    
    Provide:
    1. Diagnosis of the problem
    2. Step-by-step solution
    3. Prevention tips
    
    Be technical but clear."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {"response": response.content}

def handle_billing(state: SupportState) -> SupportState:
    """Handle billing queries."""
    print("💳 Handling billing inquiry...")
    
    prompt = f"""You are a billing specialist. Help with this inquiry:
    
    {state['query']}
    
    Provide:
    1. Clear explanation of billing policy
    2. Next steps
    3. Contact information if escalation needed
    
    Be empathetic and clear."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {"response": response.content}

def handle_general(state: SupportState) -> SupportState:
    """Handle general queries."""
    print("💬 Handling general inquiry...")
    
    prompt = f"""You are a customer service representative. Help with this question:
    
    {state['query']}
    
    Provide a helpful, friendly response with relevant information and resources."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {"response": response.content}

print("✓ All handler nodes created")

## Step 3: Create the Router Function

This function determines which node to route to based on the state:

In [ ]:
def route_query(state: SupportState) -> Literal["technical", "billing", "general"]:
    """Route to the appropriate handler based on category."""
    category = state.get("category", "general")
    print(f"🔀 Routing to: {category}")
    return category

print("✓ Router function created")

## Step 4: Build the Graph with Conditional Edges

In [ ]:
# Create the graph
graph_builder = StateGraph(SupportState)

# Add nodes
graph_builder.add_node("classify", classify_query)
graph_builder.add_node("technical", handle_technical)
graph_builder.add_node("billing", handle_billing)
graph_builder.add_node("general", handle_general)

# Add edges
graph_builder.add_edge(START, "classify")

# Add conditional edge - this is the key!
graph_builder.add_conditional_edges(
    "classify",
    route_query,
    {
        "technical": "technical",
        "billing": "billing",
        "general": "general"
    }
)

# All handlers lead to END
graph_builder.add_edge("technical", END)
graph_builder.add_edge("billing", END)
graph_builder.add_edge("general", END)

# Compile
support_graph = graph_builder.compile()

print("✓ Support routing graph compiled!")

## Visualize the Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(support_graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph structure:")
    print("START -> classify -> [technical|billing|general] -> END")

## Test the Router

Let's test with different types of queries:

In [ ]:
# Test 1: Technical query
print("\n" + "="*60)
print("TEST 1: Technical Query")
print("="*60)

result = support_graph.invoke({
    "query": "My application keeps crashing when I try to upload files larger than 10MB",
    "category": "",
    "response": "",
    "urgency": ""
})

print(f"\n📋 Category: {result['category']}")
print(f"⚠️  Urgency: {result['urgency']}")
print(f"\n💡 Response:\n{result['response']}")

In [ ]:
# Test 2: Billing query
print("\n" + "="*60)
print("TEST 2: Billing Query")
print("="*60)

result = support_graph.invoke({
    "query": "I was charged twice for my subscription this month. Can you help?",
    "category": "",
    "response": "",
    "urgency": ""
})

print(f"\n📋 Category: {result['category']}")
print(f"⚠️  Urgency: {result['urgency']}")
print(f"\n💡 Response:\n{result['response']}")

In [ ]:
# Test 3: General query
print("\n" + "="*60)
print("TEST 3: General Query")
print("="*60)

result = support_graph.invoke({
    "query": "What are your business hours?",
    "category": "",
    "response": "",
    "urgency": ""
})

print(f"\n📋 Category: {result['category']}")
print(f"⚠️  Urgency: {result['urgency']}")
print(f"\n💡 Response:\n{result['response']}")

## Exercise: Build a Content Moderator

Create a graph that:
1. Analyzes user-submitted content
2. Routes to different handlers based on content type (appropriate, needs_review, inappropriate)
3. Takes appropriate action for each type

Hint: Use conditional edges to route based on the classification!

In [ ]:
# Your code here!

class ContentState(TypedDict):
    content: str
    classification: str
    action: str

# TODO: Create classifier node
# TODO: Create handler nodes
# TODO: Create router function
# TODO: Build graph with conditional edges
# TODO: Test it


## Key Takeaways

In this notebook, you learned:

1. ✅ How to use `add_conditional_edges()` for routing
2. ✅ Creating router functions that return node names
3. ✅ Building branching workflows
4. ✅ Using state to make routing decisions
5. ✅ Creating intelligent classification and routing systems

## Next Steps

Continue to **Topic 4: Loops and Cycles** to learn how to create iterative workflows!